![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



# Collecting Data from Plants with Phidgets

This is a utility notebook to collect data from [Phidgets](https://www.phidgets.com/education/learn/projects/plant-kit/) sensors around a plant and store in an online spreadsheet (Ethercalc).

We have tried to make this notebook as friendly as possible for beginner programmers. However, it does require some attention to detail, so follow the instructions carefully. 

The notebook runs both Python code and Javascript (JS) code. It would be nice to do everything in Python, but it seems Javascript is necessary to communicate with the Phidgets in a Jupyter notebook. Fortunately, the Javascript code here is easy enough to read and you will not need to change it.

## Overview

The purpose of this notebook is to collect data from certain hardware sensors called "Phidgets" which are set up to monitor the environment of a plant. The data can be viewed directly and it is also saved in an online spreadsheet for later data analysis.

<div align="center">
<img src="images/plant2.jpg" alt="A plant with sensor" width="400"/><br>
A basement window with our plant.
</div>

There are important requirements you must meet in order to complete this setup:
    
- you need the Phidget VINT device, with three sensors attached
    - the temperature/humidity sensor (device type HUM1001_01)
    - the light sensor (device type LUX1000_0)
    - the moisture sensor (device type HUM1100)
- you will need a computer (Mac, Windows, Raspberry Pi) with a USB connection and internet access
- you will need the Chrome or Chromium web browser (Safari, Firefox, and Opera will not work)

For a step by step guide on how to assemble your Phidgets plant kit, see the following [link](https://www.phidgets.com/education/learn/projects/plant-kit/assemble/)

There are five main steps in this notebook. 

1. Setting up the gauges 
2. Setting up an online spreadsheet
3. Connect the plant sensors to your computer
4. Setup code to post and display data
5. Start the timer, to continuously post data

***
## Step 1. Setting up the gauges

Run the cells in this notebook, **one cell at a time**. This will give you the chance to respond to any errors, and to chose a unique name for your online storage. 

**Do not select "Run All."**

We first set up some gauges to display values for temperature, humidity, soil moisture and light levels. We also add a text box to display the data as it is read in, and another to show the results of posting to the spreadsheet.  

We first load in all the libraries we need. Click on `▶Run`

In [ ]:
import json, pytz, os.path, random, re, requests, threading
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from time import sleep
from datetime import datetime
from IPython.display import display, Javascript, IFrame, Markdown
print('libraries imported')

In [ ]:
# Special libraries for the anywidgets
import pathlib
import traitlets
try:
    import anywidget
except:
    !pip install --user "anywidget[dev]"
    import anywidget

Now we set up the gauges and text boxes. This is not live data quite yet but we will be soon. 

Click on `▶Run`

In [ ]:
# the four sensor gauges
g_temp = go.Indicator(
    mode = "gauge+number",
    value = 20,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Temperature"},
    gauge = {'axis': {'range': [10, 40]}}
)

g_hum = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Humidity"},
    gauge = {'axis': {'range': [0, 100]}}
)

g_moist = go.Indicator(
    mode = "gauge+number",
    value = 0.5,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Moisture"},
    gauge = {'axis': {'range': [0, 1.0]}}
)

g_light = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Light Level"},
    gauge = {'axis': {'range': [0, 10000]}}
)

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type' : 'domain'}, {'type' : 'domain'}],[{'type' : 'domain'}, {'type' : 'domain'}]],
    vertical_spacing = 0.35
)
fig.append_trace(g_temp, row=1, col=1)
fig.append_trace(g_hum, row=1, col=2)
fig.append_trace(g_moist, row=2, col=1)
fig.append_trace(g_light, row=2, col=2)

gauges = go.FigureWidget(fig)

# a text widget to monitor the latest data
data_monitor = widgets.Text(
    description='Latest data:',
    value="No recent data yet",
    layout={'width':'500px'},
    disabled = False
)
# a text widget to monitor results of posting to the cloud
post_monitor = widgets.Text(
    description='Post result:',
    value="No post yet",
    layout={'width':'500px'},
    disabled = False
)

# the watering button
data_button = widgets.Button(
    description='Update gauges',
    tooltip='Update data to gauges',
    disabled=False,
    button_style='success',
)

# the auto-watering Start/Stop button
post_button = widgets.Button(
    description='Post to spreadsheet',
    tooltip='Post the latest data to the spreadsheet',
    disabled=False,
    button_style='success',
)
    
dashboard = widgets.VBox([gauges,widgets.HBox([data_button,data_monitor]),
                              widgets.HBox([post_button,post_monitor])])

display(dashboard)

***
## Step 2: Setting up the online spreadsheet.

The next step is to create a spreadsheet online where we will store the data. We use a web resource called **EtherCalc.** The most reliable one we have found is at https://ethercalc.nomagic.uk, which is a free, open source project. You might also investigate ethercalc.net or framacalc.org as alternative sites. 

The **base_url** variable below is where we set the web address for the ethercalc storage. 

The following code cell presents a text suggesting a random name for your spreadsheet. You can always make up your own name. If you have already used this notebook, the code will suggest using the same name as last time. But you can alway choose a new one by typing in the text box.

Note this spreadsheet is publically viewable. Please **avoid storing any confidential or sensitive data in it**.You should choose a name that is meaningful for you. 

Click on `▶Run`

In [ ]:
base_url = 'https://ethercalc.nomagic.uk/'  # the online spreadsheet home page
file_name = './ss_name.txt'  # the file where we store the spreadsheet name

def get_ss_name():
    if (os.path.isfile(file_name)):
        with open(file_name,'r') as file:
            result=json.load(file)
        if result['name'] != 'default':
            return result
    json_name = {"name" : "Plant_" + "".join(random.choices('0123456789', k=6))}
    with open(file_name,'w') as file:
        json.dump(json_name,file)
        return json_name
    
def set_ss_name(name):
    with open(file_name,'w') as file:
        json.dump({"name" : name},file)
    
def f(SS_Name):
    if len(SS_Name) == 0:
        return "Enter a name here for your spreadsheet."
    if re.search(r'[^A-Za-z0-9_\-]',SS_Name):
        return "ERROR: Use only letters and numbers, no spaces."
    else:
        set_ss_name(SS_Name)
        return "Spreadsheet name is " + SS_Name

print("Choose a name for your spreadsheet here:")
widgets.interact(f, SS_Name=get_ss_name()['name']);

### 2a. Initializing the spreadsheet

Once you have selected a name above, we store some header information to the online spreadsheet to ensure that it works. 

The following code creates some headers and posts it online. It will give a status code to indicate whether the post was successful. 

Click on `▶Run`

In [ ]:
post_url = base_url + '_/' + get_ss_name()['name']

command_list = [
    'set A1 text t Date',
    'set B1 text t Time',
    'set C1 text t Temperature',
    'set D1 text t Humidity',
    'set E1 text t Moisture',    
    'set F1 text t Luminance',
    'pane row 2'
]

for command in command_list:
    myJSON = {'command':command}
    r = requests.post(post_url, json= myJSON)
    if r.status_code == 202:
        print(f"Command '{command[:6]}' successful")
    else:
        print(f"Command '{command}' failed")

### 2b. Viewing the spreadsheet

Once your spreadsheet has some data in it, it will look something like this:

<img src="images/SS2.png">
    
The following code cells will access the spreadsheet you created yourself above.

`▶Run` this next cell to see your spreadsheet.

In [ ]:
site_url = base_url + get_ss_name()['name'] + '/view'
IFrame(site_url, 700,500)

`▶Run` the following code to see the name of your spreadsheet and view it in another window.

In [ ]:
site_url = base_url + get_ss_name()['name'] + '/view'
Markdown(f"We can open your spreadsheet in a separate browser tab by clicking on this link: \n \
\n \
<a href={site_url} target='_blank'>{site_url}</a>")

***
## Step 3: Connecting the sensor devices

At this point, you should connect the Phidgets hardware to your computer. This includes 4 separate devices:
- the Phidget VINT device, attached to the computer's USB port
- the combined temperature/humidity sensor, attached to the VINT
- the soil moisture sensor, attached to the VINT
- the light sensor, attached to the VINT

When you first attach the VINT device to the USB port, your computer may ask you (pop up) whether you wish to connect to this USB device. Please answer **"yes"** to this security request. 

We now go set up the software to communicate with the Phidget devices.

### 3a. Creating the Phidget widget

We create a widget to control the phidgets. This widget is called "PhidgetFour_read" and is designed to deal with the four phidget sensors (temperature, humidity, moisture and light level) that come with the Phidget plant kit.

Once created, the Phidget widget is then connected to the gauges and the buttons in the user interface.

However, you will not see the Phidget widget on the screen until step 3b.
Click on `▶Run`

In [ ]:
## define the class
class PhidgetFour_read(anywidget.AnyWidget):
    _esm = pathlib.Path("ph4read.js")
    _css = pathlib.Path("ph4xx.css")
    # These 5 variables are used by the widget class. Do not delete. 
    hubPort = traitlets.Int(0).tag(sync=True)   
    temperature = traitlets.Float(0).tag(sync=True)
    humidity = traitlets.Float(0).tag(sync=True)
    moisture = traitlets.Float(0).tag(sync=True)
    luminance = traitlets.Float(0).tag(sync=True)

## create the PhidgetFour widget and name it pf
pf = PhidgetFour_read()

# connect the Phidget widget to the gauges in the user interface
def updateTemperature(change):
    gauges.data[0]['value'] = change.new
def updateHumidity(change):
    gauges.data[1]['value'] = change.new
def updateMoisture(change):
    gauges.data[2]['value'] = change.new
def updateLuminance(change):
    gauges.data[3]['value'] = change.new
pf.observe(updateTemperature, names=["temperature"])
pf.observe(updateHumidity, names=["humidity"])
pf.observe(updateMoisture, names=["moisture"])
pf.observe(updateLuminance, names=["luminance"])

## Now we connect the data button in the user interface to the Phidget widget named pf
def update_gauges(button):
    pf.send("read")
data_button.on_click(update_gauges)

### 3b. Opening the USB connection

Steps connecting your USB device to your Phidget VINT device to your computer. 
1. The VINT must be plugged into your computer's USB port. 
1. Click on `▶Run` in the following cell
1. You will see two buttons. Click on the first one, to connect the Phidget devices
1. Follow the prompts to select the VINT device (a list appears that you should click on). This will pair the device with your computer. 

In [ ]:
pf

This should open a window that asks you to select the Phidget Hub. Do so, and click "OK." The window looks like this:

<img src="images/Pconnect.png" width=300>

### 3c. Confirm the USB connection 

The second button above will read the data. 
The status line will tell you if the USB and devices are connected.

Click on the second button to read the data. 

If nothing happens, make sure you have the VINT hub connected to your computer's USB port. You should also have the sensors connected to the VINT hub. 

Click "Disconnect" and then "Connect" again. Try reading the data a second time. Sometimes the Phidgets need a few seconds to connect properly, so disconnecting and connecting again can help. 

## Step 4: Setup code to display and post the data

The following code grabs data from the gauges and posts it onto the online spreadsheet. In the text boxes, it will also display what data was collected and the result of the online post. Results codes like 200 or 202 are good. 

TIME ZONE CHANGE: If you like, you can change the time zone 'Canada/Pacific' below to your time zone. For instance, for Alberta, change it to 'Canada/Mountain'

The code is connected to a button on the display, so it posts whenever you click the button.


Click on `▶Run`

In [ ]:
# set the time zone
tz = pytz.timezone('Canada/Pacific')

# here we grab the data results from gauges (g) then post. 
def post_stuff(button):
    data = datetime.now(tz).strftime("%Y-%m-%d,%H:%M:%S")
    data += ', ' + f'{gauges.data[0]["value"]:.2f}'
    data += ', ' + f'{gauges.data[1]["value"]:.2f}'
    data += ', ' + f'{gauges.data[2]["value"]:.2f}'
    data += ', ' + f'{gauges.data[3]["value"]:.2f}'
    data_monitor.value = data
    post_url = base_url + '_/' + get_ss_name()['name']
    r = requests.post(post_url, data= data) ## we could return a status code here...
    post_monitor.value = datetime.now(tz).strftime("%Y-%m-%d,%H:%M:%S") + ", Status: " + str(r.status_code)

post_button.on_click(post_stuff)

The dashboard is now ready to run. The buttons will now work, to update the gauges and to post data.

In [ ]:
display(dashboard)

## YAY. We are all done

The data will be updated each time you click the "Update gauges" button. To post this new data, click the "Post to spreadsheet" button. 

You can  see the results in the text boxes, and check the online spreadsheet. The following cell shows the current spreadsheet, which will update when you click the "Post to spreadsheet" button.

In [ ]:
site_url = base_url + get_ss_name()['name'] + '/view'
IFrame(site_url, 700,500)

***
## Step 5. Going further with automation

We have the users to manually update the gauges, as this is to make sure the data is up to date Jupyter Hubs. 

But you can automate this and we have the gauges update automatically using a timer in Python but **not always reliable**. 

To have the data continuously updated, we need change the following code  line **"test_code = False" to "test_code = True"**

If you are experiencing issues, you may need to restart the kernel and change the test code back to False.

Click on `▶Run`

In [ ]:
test_code = True

def post_loop(g):
    total = 100*24*4 ## post for 100 days (4 per hour gives 100*24*4 total)
    for i in range(total):
        if stop_thread:
            break
        update_gauges(0)
        sleep(2)
        post_stuff(0)
        sleep(10) ## sleep(15*60-3)  ## Sleep for 15 minutes before next post
        
if test_code:
    stop_thread = False
    thread = threading.Thread(target=post_loop,args=(gauges,))  
    thread.start()

***
## Step 6. Finishing up: Closing down the sensors.

Once you are done with the Phidgets, you should close them down. This makes the devices available to other software or other users of the computer.

To disconnect, click on the Phidget widget button above that says "Disconnect."

Or, simply close this notebook. The Phidgets will then automatically disconnect.

## Conclusion

*We have demonstrated the process of running a utility that retrieves Phidget sensor data and publishes it on the web. This operation involves a combination of Javascript and Python to interact with both the front-end and back-end components within the Jupyter notebook environment.*

If you plan to collect data over an extended period, it's essential to have the Jupyter "Hub" software installed on your local computer. Please note that if you run this software on the Callysto hub (https://hub.callysto.ca), it may automatically disconnect after approximately a day of continuous data collection.

### Next Notebook
You may now go to the next notebook to do the data analysis, here: 
[plants-analysis.ipynb](plants-analysis.ipynb)

***
For additional support or inquiries related to this notebook, feel free to reach out to M. Lamoureux or Mary Grant, who serve as Callysto ambassadors. They can provide further details and assistance.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)